In [1]:
import pandas as pd
import numpy as np
import string
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from nltk.stem import PorterStemmer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [2]:
df=pd.read_csv('FAANG_STOCK_NEWS_Dataset.csv')

In [3]:
df.head()

,Unnamed: 0,ticker,date,time,headline,neg,neu,pos,compound,Unnamed: 9
0,0,FB,22-09-2021,06:30AM,2 Growth Stocks to Buy Hand Over Fist If the M...,0.000,0.608,0.392,0.7003,NaN
1,1,FB,22-09-2021,02:23AM,UPDATE 1-Facebook wraps up deals with Australi...,0.167,0.833,0.000,-0.3400,NaN
2,2,FB,22-09-2021,12:36AM,Facebook wraps up deals with Australian media ...,0.179,0.821,0.000,-0.3400,NaN
3,3,FB,22-09-2021,12:27AM,Facebook wraps up deals with Australia media f...,0.000,1.000,0.000,0.0000,NaN
4,4,FB,21-09-2021,10:34PM,Facebook overpaid FTC fine as quid pro quo to ...,0.099,0.659,0.242,0.3818,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  500 non-null    int64  
 1   ticker      500 non-null    object 
 2   date        500 non-null    object 
 3   time        500 non-null    object 
 4   headline    500 non-null    object 
 5   neg         500 non-null    float64
 6   neu         500 non-null    float64
 7   pos         500 non-null    float64
 8   compound    500 non-null    float64
 9   Unnamed: 9  0 non-null      float64
dtypes: float64(5), int64(1), object(4)
memory usage: 39.2+ KB


In [5]:
filtered_df = df[df['ticker'] == 'AMZN']
# Print the unique ticker values
print(len(filtered_df))

100


In [6]:
# Copy of df
cdf=df.copy()

In [7]:
# Function to classify sentiment

# When analyzing the compound score, I discovered that approximately values less than -0.1 are typically classified as negative sentiment,
# while values greater than 0.4 are considered positive sentiment. The rest of the values are typically categorized as neutral sentiment.

Sentiment = [] # For storing positive,negative,neutral Sentiment.

for score in cdf['compound']:
    if score <= -0.1:
        Sentiment.append('Negative')
    elif score >= 0.4:
        Sentiment.append('Positive')
    else:
        Sentiment.append('Neutral')


In [8]:
# Add Sentiment column to DataFrame
cdf['Sentiment'] = Sentiment


In [9]:
cdf.head(20)

,Unnamed: 0,ticker,date,time,headline,neg,neu,pos,compound,Unnamed: 9,Sentiment
0,0,FB,22-09-2021,06:30AM,2 Growth Stocks to Buy Hand Over Fist If the M...,0.000,0.608,0.392,0.7003,NaN,Positive
1,1,FB,22-09-2021,02:23AM,UPDATE 1-Facebook wraps up deals with Australi...,0.167,0.833,0.000,-0.3400,NaN,Negative
2,2,FB,22-09-2021,12:36AM,Facebook wraps up deals with Australian media ...,0.179,0.821,0.000,-0.3400,NaN,Negative
3,3,FB,22-09-2021,12:27AM,Facebook wraps up deals with Australia media f...,0.000,1.000,0.000,0.0000,NaN,Neutral
4,4,FB,21-09-2021,10:34PM,Facebook overpaid FTC fine as quid pro quo to ...,0.099,0.659,0.242,0.3818,NaN,Neutral
5,5,FB,21-09-2021,06:20PM,Is GBTC Stock A Good Buy As Bitcoin Slumps Aft...,0.000,0.775,0.225,0.4404,NaN,Positive
6,6,FB,21-09-2021,04:54PM,Facebook Oversight Board to Review Special Rul...,0.000,0.769,0.231,0.4019,NaN,Positive
7,7,FB,21-09-2021,04:37PM,Facebook says Wall Street Journal report alleg...,0.000,1.000,0.000,0.0000,NaN,Neutral
8,8,FB,21-09-2021,04:36PM,These Are The 5 Best Stocks To Buy And Watch Now,0.000,0.682,0.318,0.6369,NaN,Positive
9,9,FB,21-09-2021,03:08PM,Facebook Questioned by Oversight Board Over Ce...,0.167,0.833,0.000,-0.1027,NaN,Negative


In [10]:
# Downloading NLTK resources (stopwords and punkt tokenizer)

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [11]:
# Tokenization

def clean_text(text):
    # Remove special characters, punctuation, and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Convert text to lowercase
    text = text.lower()

    # Tokenization
    tokens = word_tokenize(text)

    # Remove extra whitespaces
    text = re.sub(' +', ' ', text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    # Reassemble the tokens into a single string
    text = ' '.join(tokens)


    return text

cdf['Processed_headline'] = cdf['headline'].apply(clean_text)



In [12]:
# Vectorization

# Create the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the documents
X = tfidf_vectorizer.fit_transform(cdf['Processed_headline'])


In [13]:
# Pritning  Vectorization Features

feature_names = tfidf_vectorizer.get_feature_names_out()
print("Feature Names:", feature_names)

Feature Names: ['aapl' 'abort' 'abram' ... 'zack' 'zero' 'zuckerberg']


In [14]:
# Assigning the 'Sentiment' column as the target variable y
y=cdf['Sentiment']
y

0      Positive
1      Negative
2      Negative
3       Neutral
4       Neutral
         ...   
495    Positive
496     Neutral
497    Positive
498     Neutral
499     Neutral
Name: Sentiment, Length: 500, dtype: object

In [15]:
# Get unique sentiments and their counts
unique_sentiments = cdf['Sentiment'].value_counts()

# Print unique sentiments and their counts
print("Unique Sentiments:")
print(unique_sentiments)


Unique Sentiments:
Neutral     297
Positive    111
Negative     92
Name: Sentiment, dtype: int64


In [16]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# Using Random Forest Classifier Model
model = RandomForestClassifier(n_estimators=150, class_weight='balanced',random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(class_weight='balanced', n_estimators=150,
                       random_state=42)

In [18]:
#  Prediction using testing dataset
y_pred = model.predict(X_test)

In [19]:
# Comparing predicted values of the model when we pass testing values with the actual
# testing y values
print('Actual Sentiment Values: \n',list(y_test[20:50]))
print('Model Predicted Values using X_test: \n' ,y_pred[20:50])

Actual Sentiment Values: 
 ['Neutral', 'Negative', 'Positive', 'Neutral', 'Positive', 'Neutral', 'Positive', 'Negative', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Neutral', 'Negative', 'Negative', 'Negative', 'Neutral', 'Neutral', 'Neutral', 'Negative', 'Neutral', 'Neutral', 'Positive', 'Negative', 'Neutral', 'Neutral', 'Positive', 'Neutral', 'Neutral', 'Positive']
Model Predicted Values using X_test: 
 ['Neutral' 'Negative' 'Positive' 'Neutral' 'Positive' 'Neutral' 'Neutral'
 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral'
 'Negative' 'Negative' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral'
 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral'
 'Neutral' 'Positive']


In [20]:
# Making Model Prediction on new Unseen Sentense

new_sentence = "Texas Startups Mount Fight Against Abortion BanWithout Big Tech"
processed_sentence = clean_text(new_sentence) # processing new_sentense using our function clean_text
vectorized_sentence = tfidf_vectorizer.transform([processed_sentence]) # vectorizing

# Make predictions
prediction = model.predict(vectorized_sentence)

print("Processed Sentence:", processed_sentence)
print("Predicted Class:", prediction)

Processed Sentence: texa startup mount fight abort banwithout big tech
Predicted Class: ['Negative']


In [21]:
# Define class names
class_names = ["Positive", "Neutral", "Negative"]  # Replace with your actual class names

# Drawing Confusion matrix for checking our model accuracy
conf_matrix = confusion_matrix(y_test, y_pred)

# Display the Confusion Matrix with class names
print("Confusion Matrix:")
print(pd.DataFrame(conf_matrix, index=class_names, columns=class_names))


Confusion Matrix:
          Positive  Neutral  Negative
Positive        10       12         1
Neutral          1       54         0
Negative         0        9        13


In [22]:
# Calculating Overall Precision and Recall

TP = conf_matrix[1, 1] # True Positives
FP = conf_matrix[0, 1] # False Positives
FN = conf_matrix[1, 0] # False Negatives

# Calculate Precision and Recall
precision = TP / (TP + FP)
recall = TP / (TP + FN)

print("Precision:", precision)
print("Recall:", recall)


Precision: 0.8181818181818182
Recall: 0.9818181818181818


In [23]:
# Calculating Precision and Recall for each class i.e Positve,Negative and Neutral

from sklearn.metrics import precision_score, recall_score

# Calculate precision and recall for each class
precision_positive = precision_score(y_test, y_pred, average=None, labels=['Positive'])
recall_positive = recall_score(y_test, y_pred, average=None, labels=['Positive'])

precision_neutral = precision_score(y_test, y_pred, average=None, labels=['Neutral'])
recall_neutral = recall_score(y_test, y_pred, average=None, labels=['Neutral'])

precision_negative = precision_score(y_test, y_pred, average=None, labels=['Negative'])
recall_negative = recall_score(y_test, y_pred, average=None, labels=['Negative'])

# Print precision and recall for each class
print("Positive Class:")
print(f"Precision: {precision_positive[0]:.4f}")
print(f"Recall: {recall_positive[0]:.4f}\n")

print("Neutral Class:")
print(f"Precision: {precision_neutral[0]:.4f}")
print(f"Recall: {recall_neutral[0]:.4f}\n")

print("Negative Class:")
print(f"Precision: {precision_negative[0]:.4f}")
print(f"Recall: {recall_negative[0]:.4f}")



Positive Class:
Precision: 0.9286
Recall: 0.5909

Neutral Class:
Precision: 0.7200
Recall: 0.9818

Negative Class:
Precision: 0.9091
Recall: 0.4348


In [24]:
from sklearn.metrics import f1_score

# Assuming y_true and y_pred are your true and predicted labels, respectively
f1 = f1_score(y_test, y_pred, average='weighted')

print("F1 Score:", f1)


F1 Score: 0.7511060834590246


In [25]:
# Using cross-validation as well for checking our model accuracy

# Use StratifiedKFold for cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation and get accuracy scores
scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')

# Display the accuracy scores
print("Cross-Validation Accuracy Scores:", scores)

Cross-Validation Accuracy Scores: [0.77 0.78 0.78 0.84 0.81]


In [26]:
# Mean Accuracy score
print("Mean Accuracy:", np.mean(scores))


Mean Accuracy: 0.796


In [27]:
# Final testing using accuracy score
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.77



## With our current dataset, achieving an accuracy of 0.77 is commendable. However, there is room for improvement, which can be achieved by incorporating more data and utilizing more advanced sentiment analysis algorithms.